In [ ]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 55)

In [ ]:
df = pd.read_excel('../data/posco/xnk.xls', header=9)
try:
    iob = pd.read_excel('../data/posco/iob.xlsx', header=9, skipfooter=10, sheet_name='IOB 012021')
except ValueError:
    print('No sheet available')

iob = iob.rename(columns={
    'Unnamed: 13': 'Qty',
    'Unnamed: 0': 'Items'
})[['Items', 'Qty']]
iob = iob.groupby(['Items'],as_index=False)[['Qty']].sum()

In [ ]:
len(pd.unique(iob['Items']))
iob

In [ ]:
'''
Take out E21, B13, A42 items
'''

e21 = df.loc[df['Mã loại hình'] == 'E21', ['Mã NPL/SP', 'Ngày ĐK', 'Đơn vị tính', 'Tổng số lượng']]
b13 = df.loc[df['Mã loại hình'] == 'B13', ['Mã NPL/SP', 'Ngày ĐK', 'Đơn vị tính', 'Tổng số lượng']]
a42 = df.loc[df['Mã loại hình'] == 'A42', ['Mã NPL/SP', 'Ngày ĐK', 'Đơn vị tính', 'Tổng số lượng']]

# Rename Columns
for df in [e21, b13, a42]:
    df.rename(columns={
        'Mã NPL/SP': 'Items',
        'Ngày ĐK': 'Date',
        'Đơn vị tính': 'Unit',
        'Tổng số lượng': 'Qty'
    }, inplace=True)

In [ ]:
print(len(pd.unique(e21['Items'])))
base_report = e21.drop_duplicates(subset=['Items'])
base_report = base_report[['Items', 'Unit']]


In [ ]:
base_report

In [ ]:
'''
Output for production
'''
take_out_free = pd.read_excel('../data/posco/take_out_free.xls', header=6)
take_out_free = take_out_free[['Date', 'Item Code', 'Item Name', 'Unit', 'Qty', 'Weight']]
take_out_free['Date'] = pd.to_datetime(take_out_free['Date'], format="%d/%m/%Y", yearfirst=True)

take_out = pd.read_excel('../data/posco/take_out.xlsx', header=9)
# del take_out['Date']
# take_out.rename(columns={
#     'Date.1': 'Date'
# }, inplace=True)
# take_out['Date'] = pd.to_datetime(take_out['Date'], format="%d/%m/%Y", yearfirst=True)
# take_out = take_out[['Date', 'Item Code', 'Item Name', 'Unit', 'Qty', 'Weight']]

output_for_production = pd.concat([take_out, take_out_free])
output_for_production.reset_index(inplace=True)
del output_for_production['index'], take_out_free
output_for_production.rename(columns={
    'Item Code' : 'Items'
}, inplace=True)

# output_for_production['Qty'] = np.where(
#     (
#         (output_for_production['Unit'] == 'KG') &
#         (
#             (output_for_production['Weight'].notnull()) |
#             (output_for_production['Weight'] != 0)
#         )
#     ), 
#     output_for_production['Weight'], 
#     output_for_production['Qty']
# )

In [ ]:
take_out.columns

In [ ]:
base_report = pd.merge(
    base_report, output_for_production[['Items', 'Item Name']].drop_duplicates(subset=['Items']).rename(
        {'Unit': 'Output Unit'}
    ), 
    how='left', left_on='Items', right_on='Items'
)
# base_report
# output_for_production

In [ ]:
base_report = base_report.drop_duplicates(subset=['Items'])
base_report

In [ ]:
'''
Return
'''
return_df = pd.read_excel('../data/posco/return.xlsx', header=6)
return_df = return_df[['Date', 'Item Code', 'Unit', 'Return Qty', 'Return Weight']]
return_df['Date'] = pd.to_datetime(return_df['Date'], format="%Y%m%d", yearfirst=True)

# aggregation_functions = {
#     'Item Name': 'first',
#     'Unit': 'first',
#     'Return Qty': 'sum',
#     'Return Weight': 'sum'
# }
# return_df['Item Code'] = return_df['Item Code'].str.replace('$', '', regex=True)
# return_df = return_df.groupby(return_df['Item Code']).aggregate(aggregation_functions)

return_df.rename(columns={
    'Item Code' : 'Items',
    'Return Qty': 'Qty',
    'Return Weight': 'Weight'
}, inplace=True)
return_df.fillna(0, inplace=True)
return_df['Weight'].replace('-', 0, inplace=True)
return_df['Weight'].replace(' ', 0, inplace=True)
return_df['Weight'] = [float(x) for x in return_df['Weight']]
# return_df['Qty'] = np.where(
#     (
#         (return_df['Unit'] == 'KG') &
#         (
#             (return_df['Weight'].notnull()) |
#             (return_df['Weight'] != 0)
#         )
#     ), 
#     return_df['Weight'], 
#     return_df['Qty']
# )

In [ ]:
pd.unique(return_df['Weight'])

In [ ]:
'''
WAREHOUSE
'''
warehouse = pd.read_excel('../data/posco/iob.xlsx', header=9, skipfooter=10)
warehouse.rename(columns={
    'Unnamed: 0': 'Items',
    'Unnamed: 1': 'Name',
    'Unnamed: 3': 'Unit',
    'Unnamed: 2': 'Project',
    'Unnamed: 13': 'Qty'
}, inplace=True)
warehouse = warehouse[['Items', 'Project', 'Unit', 'Qty']]
# aggregation_functions = {
#     'Name': 'first',
#     'Unit': 'first',
#     'Qty': 'sum',
# }
# warehouse = warehouse.groupby(warehouse['Items']).aggregate(aggregation_functions)

In [ ]:
transfer = pd.read_excel('../data/posco/transfer.xlsx', header=6)
transfer['Date'] = pd.to_datetime(transfer['Date'], format="%d/%m/%Y", yearfirst=True)
transfer = transfer[['Date', 'Item Code', 'Unit', 'Transfer Qty', 'Transfer Weight']]
transfer.rename(columns={
    'Item Code': 'Items',
    'Transfer Qty': 'Qty',
    'Transfer Weight': 'Weight'
}, inplace=True)
# transfer['Qty'] = np.where(
#     (
#         (transfer['Unit'] == 'KG') &
#         (
#             (transfer['Weight'].notnull()) |
#             (transfer['Weight'] != 0)
#         )
#     ), 
#     transfer['Weight'], 
#     transfer['Qty']
# )

In [ ]:
'''
Convert to Vietnamese Unit Measurement
'''
# for df in [output_for_production, return_df, warehouse, transfer]:
# #     trim_strings = lambda x: x.strip() if isinstance(x, str) else x
# #     return df.applymap(trim_strings)
#     df['Unit'] = df['Unit'].str.strip()
#     conditions = [
#         (df['Unit'].isin(['Bottle', 'Box', 'CAN', 'EA', 'PCS', 'PAIL'])),
#         (df['Unit'] == 'KG'),
#         (df['Unit'].isin(['SH', 'ROLL', 'COIL', 'L', 'M', 'M2'])),
#         (df['Unit'].isin(['SET', 'SUITE', 'PAIR']))
#     ]
#     values = ['Cái/Chiếc', 'Kilogam', 'Cuộn', 'Bộ']
#     df['Unit'] = np.select(conditions, values)


In [ ]:
# conditions = [
#     (transfer['Unit'].isin(['Bottle', 'Box', 'CAN', 'EA', 'PCS', 'PAIL'])),
#     (transfer['Unit'] == 'KG'),
#     (transfer['Unit'].isin(['SH', 'ROLL', 'COIL', 'M', 'M2'])),
#     (transfer['Unit'].isin(['SET', 'SUITE', 'PAIR']))
# ]
# values = ['Cái/Chiếc', 'Kilogam', 'Cuộn', 'Bộ']
# transfer['Unit'] = np.select(conditions, values)
# pd.unique(transfer['Unit'])
pd.unique(warehouse['Unit'])

In [ ]:
'''
Feburary
'''
# e21 = e21.loc[e21['Date'].dt.month == 2]
# output_for_production = output_for_production.loc[output_for_production['Date'].dt.month == 2]
# return_df = return_df.loc[return_df['Date'].dt.month == 2]


# output_for_production = output_for_production.groupby(['Items','Unit'],as_index=False)['Qty'].sum()
# output_for_production.rename(columns={
#     'Qty': 'Output for production'
# }, inplace=True)

# e21 = e21.groupby(['Items','Unit'],as_index=False)['Qty'].sum()
# e21.rename(columns={
#     'Qty': 'Import'
# }, inplace=True)

# return_df['Qty'] = [int(x) for x in return_df['Qty']]
# return_df = return_df.groupby(['Items','Unit'],as_index=False)['Qty'].sum()
# return_df.rename(columns={
#     'Qty': 'Return'
# }, inplace=True)

# balanced = pd.merge(left=e21_02, right=output_for_production_02, how='left', left_on='Items', right_on='Items')

# grouped_e21 = e21_02.groupby(["Items", "Unit"])
# test_df = [None]
# for _, item in grouped_e21:
#     item.aggregate({
#     'Qty': 'sum'
#     })
#     test_df.append(item)
# warehouse = warehouse.groupby(warehouse['Items']).aggregate(aggregation_functions)


# test_df = e21_02.groupby(['Items','Unit'],as_index=False)['Quantity'].sum()
# test_df = pd.concat(test_df)


In [ ]:
# test_report = pd.merge(
#     e21, output_for_production,  how='left', 
#     left_on=['Items','Unit'], 
#     right_on = ['Items','Unit']
# )
# test_report = pd.merge(
#     test_report, return_df,  how='left', 
#     left_on=['Items','Unit'], 
#     right_on = ['Items','Unit']
# )
# from functools import reduce

# dfs = [e21, output_for_production, return_df]
# test_report = reduce(lambda left, right: pd.merge(left, right, how='left', 
#     left_on=['Items','Unit'], 
#     right_on = ['Items','Unit']), dfs
# )


In [ ]:
# output_for_production_02
# grouped_output = output_for_production_02.groupby(["Items", "Unit"])
# output_df = [None]
# for _, item in grouped_output:
#     del item['Qty'], item['Weight']
#     item['Quantity'].sum()
#     output_df.append(item)
# output_df = pd.concat(output_df)


In [ ]:
base_report
# output_df['Quantity'] = output_df['Quantity'].transform(lambda x : sum(x))
# output_df = output_df.groupby(output_df['Items', 'Unit']).aggregate({'Quantity': 'sum'})


In [ ]:
'''
Input files: e21, b13, a42, output_for_production, return_df, warehouse, transfer
'''

In [ ]:
last_year = pd.read_excel('../data/posco/last_year.xlsx', sheet_name='BCQT.15', header=11)
# last_year = last_year[['x.1', 'x.6']]
# last_year.rename(columns={
#     'x.1': 'Items',
#     'x.6': 'Ecus stock'
# }, inplace=True)
last_year.rename(columns={
    'Unnamed: 1': 'Items',
    'Unnamed: 2': 'Item Name',
    ' - Ngày: 19/02/2020 - Ngày hết hạn: 31/12/2020': 'Unit',
    'Unnamed: 10': 'Ecus stock'
}, inplace=True)
e21_last_year = last_year[['Items', 'Item Name', 'Unit']]
# e21_last_year
base_report = base_report[['Items', 'Item Name', 'Unit']]
base_report = pd.merge(
            base_report, e21_last_year, how='outer', 
                left_on=['Items'], 
                right_on = ['Items']
        )
base_report['Item Name'] = np.where(
    (
        (base_report['Item Name_x'].notnull())
    ), 
    base_report['Item Name_x'], 
    base_report['Item Name_y']
)
base_report['Unit'] = np.where(
    (
        (base_report['Unit_x'].notnull())
    ), 
    base_report['Unit_x'], 
    base_report['Unit_y']
)
base_report.drop(['Unit_x', 'Unit_y', 'Item Name_x', 'Item Name_y'], axis=1, inplace=True)

In [ ]:
last_year = last_year[['Items', 'Ecus stock']]

In [ ]:
transfer.info()

In [ ]:
'''
Snippet for final result
'''
E21_DF = [None] * 13
B13_DF = [None] * 13
A42_DF = [None] * 13
OUTPUT_DF = [None] * 13
RETURN_DF = [None] * 13
TRANSFER_DF = [None] * 13
WAREHOUSE_DF = [None] * 13
BALANCED_REPORT = [None] * 13

from functools import reduce

for month in range(1, 13):
    E21_DF[month] = e21.loc[e21['Date'].dt.month == month]
    B13_DF[month] = b13.loc[b13['Date'].dt.month == month]
    A42_DF[month] = a42.loc[a42['Date'].dt.month == month]
    OUTPUT_DF[month] = output_for_production.loc[output_for_production['Date'].dt.month == month]
    RETURN_DF[month] = return_df.loc[return_df['Date'].dt.month == month]
    TRANSFER_DF[month] = transfer.loc[transfer['Date'].dt.month == month]
    
    E21_DF[month] = E21_DF[month].groupby(['Items'],as_index=False)[['Qty']].sum()
    E21_DF[month].rename(columns={
        'Qty': 'Import',
    }, inplace=True)
    
    B13_DF[month] = B13_DF[month].groupby(['Items'],as_index=False)[['Qty']].sum()
    B13_DF[month].rename(columns={
        'Qty': 'Re-export',
    }, inplace=True)
    
    A42_DF[month] = A42_DF[month].groupby(['Items'],as_index=False)[['Qty']].sum()
    A42_DF[month].rename(columns={
        'Qty': 'Re-purpose',
    }, inplace=True)
    
    OUTPUT_DF[month] = OUTPUT_DF[month].groupby(['Items'],as_index=False)[['Qty', 'Weight']].sum()
    OUTPUT_DF[month].rename(columns={
        'Qty': 'Output for production',
        'Weight': 'Output Weight'
    }, inplace=True)
    
    RETURN_DF[month]['Qty'] = [int(x) for x in RETURN_DF[month]['Qty']]
    RETURN_DF[month] = RETURN_DF[month].groupby(['Items'],as_index=False)[['Qty', 'Weight']].sum()
    RETURN_DF[month].rename(columns={
        'Qty': 'Return',
        'Weight': 'Return Weight'
    }, inplace=True)
    
    TRANSFER_DF[month] = TRANSFER_DF[month].groupby(['Items'],as_index=False)[['Qty', 'Weight']].sum()
    TRANSFER_DF[month].rename(columns={
        'Qty': 'Transfer',
        'Weight': 'Transfer Weight'
    }, inplace=True)
    
    
    dfs = None
    if month > 1:
        last_ecus = BALANCED_REPORT[month - 1][['Items', 'Ecus stock']]
        
        last_ecus.rename(columns={
            'Ecus stock': 'Begin'
        }, inplace=True)
        BALANCED_REPORT[month] = base_report
#         for df in dfs:
#             BALANCED_REPORT[month] = pd.merge(BALANCED_REPORT[month], df, how='left', 
#                 left_on=['Items'], 
#                 right_on = ['Items'])
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], last_ecus, how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
    
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], E21_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], B13_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], A42_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], OUTPUT_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], RETURN_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        
        BALANCED_REPORT[month].fillna(0, inplace=True)
        BALANCED_REPORT[month]['Ecus stock'] = (
        BALANCED_REPORT[month]['Begin'] + BALANCED_REPORT[month]['Import'] - BALANCED_REPORT[month]['Output for production'] - 
        BALANCED_REPORT[month]['Re-export'] - BALANCED_REPORT[month]['Re-purpose']
        )
        try:
            iob = pd.read_excel('../data/posco/iob.xlsx', header=9, skipfooter=10, sheet_name=f'IOB {month:02}2021')
            iob = iob.rename(columns={
                'Unnamed: 13': 'Qty',
                'Unnamed: 0': 'Items'
            })[['Items', 'Qty']]
            iob = iob.groupby(['Items'],as_index=False)[['Qty']].sum()
            iob.rename(columns={
                'Qty': 'Warehouse stock'
            }, inplace=True)
            BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], iob, how='left', 
                left_on=['Items'], 
                right_on = ['Items']
            )
            BALANCED_REPORT[month]['Balance'] = BALANCED_REPORT[month]['Warehouse stock'] - BALANCED_REPORT[month]['Ecus stock']
        except ValueError:
            print('No Warehouse sheet found in month ' + str(month))
        
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], TRANSFER_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
    else:
#         dfs = [E21_DF[month], B13_DF[month], A42_DF[month], OUTPUT_DF[month], RETURN_DF[month]]
#         BALANCED_REPORT[month] = reduce(lambda left, right: pd.merge(left, right, how='left', 
#             left_on=['Items'], 
#             right_on = ['Items']), dfs
#         )
        last_ecus = last_year
        
        last_ecus.rename(columns={
            'Ecus stock': 'Begin'
        }, inplace=True)
        
        BALANCED_REPORT[month] = pd.merge(
            base_report, last_ecus, how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], E21_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], B13_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], A42_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], OUTPUT_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], RETURN_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
        BALANCED_REPORT[month].fillna(0, inplace=True)
        BALANCED_REPORT[month]['Ecus stock'] = (
        BALANCED_REPORT[month]['Begin'] + BALANCED_REPORT[month]['Import'] - BALANCED_REPORT[month]['Output for production'] - 
        BALANCED_REPORT[month]['Re-export'] - BALANCED_REPORT[month]['Re-purpose']
        )
        try:
            iob = pd.read_excel('../data/posco/iob.xlsx', header=9, skipfooter=10, sheet_name=f'IOB {month:02}2021')
            iob = iob.rename(columns={
                'Unnamed: 13': 'Qty',
                'Unnamed: 0': 'Items'
            })[['Items', 'Qty']]
            iob = iob.groupby(['Items'],as_index=False)[['Qty']].sum()
            iob.rename(columns={
                'Qty': 'Warehouse stock'
            }, inplace=True)
            BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], iob, how='left', 
                left_on=['Items'], 
                right_on = ['Items']
            )
            BALANCED_REPORT[month]['Balance'] = BALANCED_REPORT[month]['Warehouse stock'] - BALANCED_REPORT[month]['Ecus stock']
        except ValueError:
            print('No Warehouse sheet found in month ' + str(month))
        
        BALANCED_REPORT[month] = pd.merge(
            BALANCED_REPORT[month], TRANSFER_DF[month], how='left', 
                left_on=['Items'], 
                right_on = ['Items']
        )
            
BALANCED_REPORT[month]['Output for production'] = np.where(
        (BALANCED_REPORT[month]['Unit'] == 'Kilogam')
    , 
        BALANCED_REPORT[month]['Output Weight'], 
        BALANCED_REPORT[month]['Output for production']
)
BALANCED_REPORT[month]['Return'] = np.where(
    (
        (BALANCED_REPORT[month]['Unit'] == 'Kilogam')
    ), 
    BALANCED_REPORT[month]['Return Weight'], 
    BALANCED_REPORT[month]['Return']
)

BALANCED_REPORT[month]['Transfer'] = np.where(
    (
        (BALANCED_REPORT[month]['Unit'] == 'Kilogam')
    ), 
    BALANCED_REPORT[month]['Transfer Weight'], 
    BALANCED_REPORT[month]['Transfer']
)

In [ ]:

    
# with pd.ExcelWriter('../output/posco_result.xlsx') as writer:  
#     for month in range(1, 13):
#         BALANCED_REPORT[month].to_excel(writer, sheet_name=f'BALANCED_{month:02}')

In [ ]:
def color_parenthesis_red(val):   
    color = 'red' if val.startswith('(') else 'black'
    return 'color: %s' % color
with pd.ExcelWriter('../output/posco_result.xlsx') as writer:  
    for df in enumerate(BALANCED_REPORT[1:], start=1):
        

        #transform values into string and add parenthesis when they are negative
#         df[1] = df[1].astype(str).apply(lambda x: f"({x})" if int(x)<0 else x)
#         df[1].style.applymap(color_negative_red)
        
        df[1].drop(['Return Weight', 'Output Weight', 'Transfer Weight'], axis=1, inplace=True)
        df[1].fillna(0, inplace=True)
        df[1].to_excel(writer, sheet_name=f'BALANCED_{(df[0]):02}')

In [ ]:
pd.unique(BALANCED_REPORT[1]['Warehouse stock'])